In [21]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)  #allows display of all columns in dfs

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

import snowflake.connector
from snowflake.sqlalchemy import URL

In [22]:
# establish Snowflake connection

# import sys
# sys.path.insert(0, '../Key')

# from snowflake_key import user, account, host


# url = URL(
#     user = user,
#     account = account,
#     host = host,
#     authenticator='externalbrowser',
#     warehouse='ISC_BI',
#     database='INFOSCOUT'
# )


In [23]:
# # create engine
# engine = create_engine(url)
# connection = engine.connect()

In [24]:
import torch
from transformers import BertForNextSentencePrediction, BertTokenizer

In [25]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForNextSentencePrediction.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
question = "Would you purchase this product again?"
# answer = "I would buy every product this company sells"
answer = "sakfljs;lfjks"
encoding = tokenizer(question, answer, return_tensors="pt")

In [20]:
outputs = model(**encoding, labels=torch.LongTensor([1]))
response_score = torch.softmax(outputs.logits, dim=1)[0, 0]
is_good_response = response_score > 0.5
print(f"response score: {response_score}, is good response: {is_good_response}")

response score: 0.0005691330297850072, is good response: False


In [32]:
df = pd.read_csv('panelist_quality_test_data.csv')
df = df.head(1000)

In [ ]:
# create empty list for holding dataframes to concatenate
dfs = []

# iterate through df
for row in df.itertuples():
    user_id = row.user_id
    question = row.question
    answer = str(row.answer)
    encoding = tokenizer(question, answer, return_tensors="pt")
    
    # get score and if response classification
    outputs = model(**encoding, labels=torch.LongTensor([1]))
    response_score = torch.softmax(outputs.logits, dim=1)[0, 0]
    is_good_response = response_score > 0.5
    
#     print(f"response score: {answer} {response_score}, is good response: {is_good_response}")
    
    temp_df = pd.DataFrame({"user_id" : [user_id],
                           "question" : question,
                           "answer" : [answer], 
                           "response_score" : float(response_score),
                           "is_good_response" : bool(is_good_response)
                           })
    
    dfs.append(temp_df) 

# combine dataframes
results_df = pd.concat(dfs, ignore_index=True, sort=False)
results_df.head()

In [ ]:
results_df.to_csv("../open_end_quality_data.csv", index=False)